In [ ]:
import pymssql
import pandas
import yaml

CONFIG_FILE_PATH = 'config/config.yaml'
config = yaml.load(open(CONFIG_FILE_PATH, 'r', encoding='utf-8'), Loader=yaml.FullLoader)

In [ ]:
def get_label_rank_df(
    model_id, # int
    dissimilarity_id, # int
    batch_size=100, # float
    relabel_lambda=0.069, # float
    random_ratio=0.5, # float
):
    
    batch_size_d = int((1- random_ratio)*batch_size)
    batch_size_r = batch_size - batch_size_d
    
    # call stored procedure 1 w/ D_ID
    args = {
        "M_ID": model_id,
        "D_ID": dissimilarity_id,
        "BATCH_SIZE": batch_size_d,
        "RELABEL_LAMBDA": relabel_lambda,
    }
    d_df = get_data_sp(sp="AL_RANKINGS", args)
    
    # call stored  procedure 1 w/ D_ID = 0 # represents random
    args['D_ID'] = 0
    args['BATCH_SIZE'] = batch_size_r
    r_df = get_data_sp(sp="AL_RANKINGS", args)
    
    full_df = pd.concat([d_df, r_df]) # might have duplicates but the chance is so low it doesn't matter
    
    return full_df
    

def get_data_sp(
    sp="", # string
    args, # dict
):
    # load from config file
    server = config['server']     # 'your_server.database.windows.net'
    database = config['database'] #'your_database'
    username = config['username'] # 'your_username'
    password = config['password'] #'your_password'

    # set up a connection to Azure
    with pymssql.connect(server, user, password, database) as conn:
        # set up a cursor object
        with conn.cursor(as_dict=True) as cursor:
            # gather variables for the stored procedure
            arg_tuples = (args['M_ID'], args['D_ID'], args['RELABEL_LAMBDA'], args['BATCH_SIZE'])
            # execute stored procedure
            cursor.callproc(sp, arg_tuples)
            
            # Fetch the results into a Pandas DataFrame
            df = pd.DataFrame(cursor.fetchall())
#             columns = [column[0] for column in cursor.description]
#             df = pd.DataFrame(cursor.fetchall(), columns=columns)
    return df
    
    

In [13]:
import pymssql

server = 'capstoneservercjault.database.windows.net'
user = 'CloudSA85da88d3'
password = 'Passwordtest123'
database = 'capstoneazure'


with pymssql.connect(server, user, password, database) as conn:
    with conn.cursor() as cursor:
        cursor.execute('SELECT @@VERSION')
#         row = cursor.fetchone()
        row = cursor.fetchall()
#         while row:
#             print("SQL Server version:", row[0])
#             row = cursor.fetchone()


ColumnsWithoutNamesError: Specified as_dict=True and there are columns with no names: [0]

In [9]:
type(row)

list

In [11]:
row[0]

('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tJan  3 2024 16:32:18 \n\tCopyright (C) 2022 Microsoft Corporation\n',)